In [1]:
from typing import Tuple, List, Union, Any, Optional, Dict, Literal, Callable
import time

from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
from torch import Tensor, tensor
from scipy.optimize import minimize

from models.sandwiched_least_squares import sandwiched_LS_scalar, sandwiched_LS_diag, sandwiched_LS_dense

# Scalar case

Let $R \in R^{n \times d}, W\in R^{D\times d}, \Delta \in R,$ and $X \in R^{n \times D}$. Let $\lambda > 0$. Then the minimum of 
\begin{align*}
    J(\Delta) = \frac{1}{n} \sum_{i=1}^n \big\| R_i - W^\top\Delta X_i \big\|^2 + \lambda \Delta^2
\end{align*}
is uniquely attained by
\begin{align*}
    \Delta_{\textnormal{scalar}} 
    &= \frac{\langle R, XW^\top\rangle_F}{\|X W\|_F^2 + n\lambda} 
    = \frac{\frac{1}{n}\sum_{i=1}^n \langle W^\top X_i,  R_i\rangle}{\frac{1}{n}\sum_{i=1}^n \|W^\top X_i\|^2 + \lambda }.
\end{align*}

In [2]:
# Parameters
d = 200
D = 300
N = 100
l2_reg = 10

# Create dummy data
np.random.seed(0)
r = np.random.randn(N, d)+2
W = np.random.randn(D, d)/100
x = np.random.randn(N, D)-1

def J(Delta):
    Wx = W.T @ x.T
    residual = r - Wx.T * Delta
    return np.mean(np.linalg.norm(residual, axis=1)**2) + l2_reg * Delta**2

def J_byhand(Delta):
    res = 0
    for i in range(N):
        res += np.linalg.norm(r[i] - W.T @ x[i] * Delta)**2 / N
    return res + l2_reg * Delta**2

In [3]:
# Closed form solution
Delta_closed_form = sandwiched_LS_scalar(tensor(r), tensor(W), tensor(x), l2_reg).numpy()

print(f"Closed form solution for Delta: {Delta_closed_form}")
print(f"Objective value for closed form solution: {J(Delta_closed_form)}")
print(f"Objective value for closed form solution (by hand): {J_byhand(Delta_closed_form)}")

Closed form solution for Delta: 0.09612819692159835
Objective value for closed form solution: 993.0117548905527
Objective value for closed form solution (by hand): 993.0117548905528


In [4]:
# Gradient descent solution using scipy.optimiz
result = minimize(J, np.random.randn(), method='BFGS')
Delta = result.x[0]
print(f"Gradient descent solution for Delta using scipy.optimize: {Delta}")
print(f"Objective value for gradient descent solution: {J(Delta)}")
print(f"Objective value for closed form solution (by hand): {J_byhand(Delta)}")

Gradient descent solution for Delta using scipy.optimize: 0.09612803569713149
Objective value for gradient descent solution: 993.0117548905532
Objective value for closed form solution (by hand): 993.0117548905529


# Diagonal Case

Let $R \in R^{n \times d}, W\in R^{D\times d}, \Delta = \textnormal{diag}(\delta_1, ..., \delta_D) \in R^{D \times D},$ and $X \in R^{n \times D}$. Let $\lambda > 0$. Then the minimum of 
\begin{align*}
    J(\Delta) = \frac{1}{n} \sum_{i=1}^n \big\| R_i - W^\top\Delta X_i \big\|^2 + \lambda \Delta^2
\end{align*}
is uniquely attained by the solution to the system of linear equations
\begin{align*}
    b = (A+ \lambda I)\Delta
\end{align*}
where
\begin{align*}
    A = W W^\top \odot X^\top X,  \qquad \qquad b = \textnormal{diag}(W^\top R^\top X).
\end{align*}


In [5]:
# Parameters
d = 30
D = 20
N = 1000
l2_reg = 10

# Create dummy data
np.random.seed(0)
r = np.random.randn(N, d)
W = np.random.randn(D, d)
x = np.random.randn(N, D)

def A_byhand():
    A = np.zeros((D, D))
    for k in range(D):
        for j in range(D):
            A[k, j] = np.mean([ x[i, k] * x[i, j] * np.dot(W[k], W[j]) for i in range(N)])
    return A


def A():
    return (W @ W.T) * (x.T @ x) / N

def b_byhand():
    b = np.zeros(D)
    for k in range(D):
        b[k] = np.mean([ x[i, k] * np.dot(W[k], r[i]) for i in range(N)])
    return b


def b():
    return np.mean( (r @ W.T) * x, axis=0)
    #return np.diag(W @ r.T @ x) / N
    #return np.einsum('nd,kd,nk->k', r, W, x) / N

In [6]:
(A() - A_byhand()).mean()

-2.0510801173809213e-17

In [7]:
(b() - b_byhand()).mean()

2.4134340359527328e-17

In [8]:
def J(Delta):
    return np.mean(np.linalg.norm(r - x @ np.diag(Delta) @ W, axis=1)**2) + l2_reg * np.linalg.norm(Delta)**2

def J_byhand(Delta):
    res = 0
    for i in range(N):
        res += np.linalg.norm(r[i] - W.T @ (Delta*x[i]))**2 / N
    return res + l2_reg * np.sum(Delta**2)

In [9]:
Delta_closed_form = sandwiched_LS_diag(tensor(r), tensor(W), tensor(x), l2_reg).numpy()
print(f"Closed form solution for Delta: {Delta_closed_form}")
print(f"Objective value for closed form solution: {J(Delta_closed_form)}")
print(f"Objective value for closed form solution (by hand): {J_byhand(Delta_closed_form)}")

Closed form solution for Delta: [ 1.15952169e-03  5.00119337e-03  8.21619145e-03  1.07988685e-02
 -2.68865051e-03  4.19528345e-03 -4.25148869e-03  5.80865727e-04
  4.97751029e-03 -7.35732222e-03  2.18509303e-06 -6.20406520e-03
  3.83358855e-03  8.42855673e-04 -5.64821215e-03  6.42419030e-03
  3.03962442e-03  1.63056427e-03  5.86754698e-03  7.41961758e-03]
Objective value for closed form solution: 29.54087641951766
Objective value for closed form solution (by hand): 29.540876419517648


In [10]:
# Gradient descent solution using scipy.optimiz
result = minimize(J, np.random.randn(D), method='BFGS')
Delta = result.x
print(f"Gradient descent using scipy.optimize: {Delta}")
print(f"Objective value for gradient descent solution: {J(Delta)}")
print(f"Objective value for closed form solution (by hand): {J_byhand(Delta)}")

Gradient descent using scipy.optimize: [ 1.15952293e-03  5.00119841e-03  8.21620435e-03  1.07988851e-02
 -2.68865000e-03  4.19529645e-03 -4.25149147e-03  5.80868866e-04
  4.97750987e-03 -7.35732630e-03  2.19717982e-06 -6.20406044e-03
  3.83358796e-03  8.42863595e-04 -5.64819249e-03  6.42419110e-03
  3.03962480e-03  1.63057012e-03  5.86755304e-03  7.41962085e-03]
Objective value for gradient descent solution: 29.540876419517705
Objective value for closed form solution (by hand): 29.54087641951767


# Dense Case

Let $R \in R^{n \times d}, W\in R^{D\times d}, \Delta = \in R^{D \times D},$ and $X \in R^{n \times D}$. Let $\lambda > 0$. Then the minimum of 
\begin{align*}
    J(\Delta) 
        &= \frac{1}{n} \sum_{i=1}^n \big\| r_i - W^\top \Delta x_i \big\|^2 + \sum_{k=1}^D\sum_{j=1}^p \lambda \Delta_{k,j}^2 \\
        &= \frac{1}{n}\| W^\top \Delta X^\top - R^\top\|^2_F + \lambda \|\Delta\|^2_F
\end{align*}
is uniquely obtained by solving the system of linear equations given by
\begin{align*} 
    W R^\top X    =  W W^\top \Delta X^\top X + \lambda n \Delta
\end{align*}
which can be solved by spectral decomposition $W W^\top = U \Lambda^W U^\top$  and $X^\top X = V \Lambda^X V^\top$
\begin{align*}
    \Delta_{\textnormal{dense}} = U \bigg[ U^\top W R^\top X V \oslash \bigg(\lambda N 1 + \textnormal{diag}(\Lambda^W) \otimes \textnormal{diag}(\Lambda^X)\bigg) \bigg] V^\top
\end{align*}
where $\oslash$ denotes element-wise division, $\otimes$ is the outer product, and $1$ is a matrix of ones.


In [27]:
# Parameters
d = 1
D = 50
p = 32
N = 580
l2_reg = 1

# Create dummy data
np.random.seed(0)
r = np.random.randn(N, d)
W = np.random.randn(D, d)
x = np.random.randn(N, p)

def J(Delta):
    Delta = Delta.reshape(D, p)
    return 1/N * np.linalg.norm(W.T @ Delta @ x.T - r.T)**2 + l2_reg * np.linalg.norm(Delta)**2

def J_byhand(Delta):
    Delta = Delta.reshape(D, p)
    res = 0
    for i in range(N):
        res += 1/N * np.linalg.norm(r[i] - W.T @ Delta @ x[i])**2
    return res + l2_reg * np.linalg.norm(Delta)**2

In [38]:
Delta_closed_form = sandwiched_LS_dense(tensor(r), tensor(W), tensor(x), l2_reg).numpy().T
print(f"Closed form solution for Delta: {Delta_closed_form}")
print(f"Objective value for closed form solution: {J(Delta_closed_form)}")
print(f"Objective value for closed form solution (by hand): {J_byhand(Delta_closed_form)}")

Closed form solution for Delta: [[ 5.85719218e-03  2.78400422e-04  1.54524397e-03 ... -4.94987842e-03
   3.42207227e-03 -2.37605450e-03]
 [ 1.98027695e-03  9.41252945e-05  5.22436507e-04 ... -1.67352032e-03
   1.15697942e-03 -8.03327912e-04]
 [ 3.42044677e-03  1.62578552e-04  9.02381991e-04 ... -2.89059930e-03
   1.99840054e-03 -1.38755357e-03]
 ...
 [-5.01435285e-03 -2.38339106e-04 -1.32288617e-03 ...  4.23759989e-03
  -2.92964228e-03  2.03414457e-03]
 [ 1.68450250e-04  8.00667267e-06  4.44405314e-05 ... -1.42356308e-04
   9.84172809e-05 -6.83342737e-05]
 [ 4.15639125e-06  1.97559008e-07  1.09653880e-06 ... -3.51254162e-06
   2.42837707e-06 -1.68610006e-06]]
Objective value for closed form solution: 0.9296631579761025
Objective value for closed form solution (by hand): 0.929663157976103


In [36]:
# Gradient descent solution using scipy.optimiz
result = minimize(J, np.random.randn(p*D), method='L-BFGS-B')
Delta = result.x.reshape(p, D)
print(f"Gradient descent using scipy.optimize: {Delta}")
print(f"Objective value for gradient descent solution: {J(Delta)}")
print(f"Objective value for closed form solution (by hand): {J_byhand(Delta)}")

Gradient descent using scipy.optimize: [[ 7.02715478e-03 -9.90405350e-05  8.76929837e-04 ... -7.35360974e-04
  -7.19459422e-04 -1.44819791e-03]
 [ 5.77630415e-04  1.34199067e-04  9.08216802e-04 ...  4.90763365e-05
  -3.48332652e-04 -1.01943524e-03]
 [ 7.52060808e-04  9.76043549e-04 -2.63956015e-04 ...  1.22332634e-04
   7.69396742e-04 -2.63358556e-04]
 ...
 [-2.18104905e-03 -2.18332986e-03  3.63997060e-04 ...  7.52541260e-05
   1.97727266e-04 -2.41687793e-04]
 [-1.68772141e-04  4.50067658e-04 -3.45096999e-04 ... -8.21566017e-05
   2.87504154e-05  3.39668430e-05]
 [-3.88033797e-05 -5.76620782e-05 -6.28328004e-05 ...  7.95272973e-06
  -5.95294832e-06  8.95682604e-06]]
Objective value for gradient descent solution: 0.9457129365470255
Objective value for closed form solution (by hand): 0.9457129365470257
